In [1]:
from bioblend.galaxy import GalaxyInstance
import os

In [2]:
gi = GalaxyInstance(url='https://usegalaxy.eu', key=os.environ['my_galaxy_api'])

print(gi)

GalaxyInstance object for Galaxy at https://usegalaxy.eu


In [3]:
gi.histories.get_histories()

[{'model_class': 'History',
  'id': 'ecfc8c06ceae22af',
  'name': 'shotgun paired test run',
  'deleted': False,
  'purged': False,
  'url': '/api/histories/ecfc8c06ceae22af',
  'published': False,
  'annotation': None,
  'tags': [],
  'update_time': '2023-01-24T17:43:51.334962'},
 {'model_class': 'History',
  'id': 'f9fac8fec6c258e8',
  'name': 'Unnamed history',
  'deleted': False,
  'purged': False,
  'url': '/api/histories/f9fac8fec6c258e8',
  'published': False,
  'annotation': None,
  'tags': [],
  'update_time': '2023-01-08T19:21:43.756351'},
 {'model_class': 'History',
  'id': '0124e50f643eb2b4',
  'name': 'kraken paper',
  'deleted': False,
  'purged': False,
  'url': '/api/histories/0124e50f643eb2b4',
  'published': False,
  'annotation': None,
  'tags': [],
  'update_time': '2023-01-24T14:20:25.570624'},
 {'model_class': 'History',
  'id': 'a815f9b6bd4ea13a',
  'name': 'beer tutorial',
  'deleted': False,
  'purged': False,
  'url': '/api/histories/a815f9b6bd4ea13a',
  'publ